Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel $\rightarrow$ Restart) and then **run all cells** (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "Tatiana Gonzalez"
COLLABORATORS = "Deeya Patel"

In [2]:
pip install pmdarima

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pandas as pd
from scipy.stats import beta
from pmdarima import auto_arima
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import sqlite3
import requests
from bs4 import BeautifulSoup

---

## Read Data

In [4]:
# read csv file
df = pd.read_csv('california_infectious_diseases.csv')
df.head(10)

,Disease,County,Year,Sex,Cases,Population,Rate,Lower_95__CI,Upper_95__CI
0,Anaplasmosis,Alameda,2001,Female,0.0,746596,-,0.0,0.494
1,Anaplasmosis,Alameda,2001,Male,0.0,718968,-,0.0,0.513
2,Anaplasmosis,Alameda,2001,Total,0.0,1465564,-,0.0,0.252
3,Anaplasmosis,Alameda,2002,Female,0.0,747987,-,0.0,0.493
4,Anaplasmosis,Alameda,2002,Male,0.0,720481,-,0.0,0.512
5,Anaplasmosis,Alameda,2002,Total,0.0,1468468,-,0.0,0.251
6,Anaplasmosis,Alameda,2003,Female,0.0,747441,-,0.0,0.494
7,Anaplasmosis,Alameda,2003,Male,0.0,719746,-,0.0,0.513
8,Anaplasmosis,Alameda,2003,Total,0.0,1467187,-,0.0,0.251
9,Anaplasmosis,Alameda,2004,Female,0.0,746723,-,0.0,0.494


## Data Cleaning

In [5]:
print("Missing Values Before Cleaning")
print(df.isna().sum())

Missing Values Before Cleaning
Disease            0
County             0
Year               0
Sex                0
Cases           4754
Population         0
Rate               0
Lower_95__CI    5271
Upper_95__CI    5271
dtype: int64


### Remove Rows Where County = 'California'

In [6]:
df = df[df['County'] != 'California'].reset_index(drop=True)

#### If Rate is '-', Calculate It

##### Formula: (Cases * 100,000) / Population

In [7]:
print("Sum of '-' Before: ", (df['Rate'] == "-").sum())
df.loc[df['Rate'] == '-', 'Rate'] = (df['Cases']*100000)/df['Population']
print("Sum of '-' After: ", (df['Rate'] == "-").sum())
df.head(10)

Sum of '-' Before:  146078
Sum of '-' After:  0


,Disease,County,Year,Sex,Cases,Population,Rate,Lower_95__CI,Upper_95__CI
0,Anaplasmosis,Alameda,2001,Female,0.0,746596,0.0,0.0,0.494
1,Anaplasmosis,Alameda,2001,Male,0.0,718968,0.0,0.0,0.513
2,Anaplasmosis,Alameda,2001,Total,0.0,1465564,0.0,0.0,0.252
3,Anaplasmosis,Alameda,2002,Female,0.0,747987,0.0,0.0,0.493
4,Anaplasmosis,Alameda,2002,Male,0.0,720481,0.0,0.0,0.512
5,Anaplasmosis,Alameda,2002,Total,0.0,1468468,0.0,0.0,0.251
6,Anaplasmosis,Alameda,2003,Female,0.0,747441,0.0,0.0,0.494
7,Anaplasmosis,Alameda,2003,Male,0.0,719746,0.0,0.0,0.513
8,Anaplasmosis,Alameda,2003,Total,0.0,1467187,0.0,0.0,0.251
9,Anaplasmosis,Alameda,2004,Female,0.0,746723,0.0,0.0,0.494


### Calculate the Exact Clopper-Pearson When Upper and Lower CI are NaN

In [8]:
# method used in dataset calculation to get lower and upper 95 for NaN values
def clopper_pearson_exact_method(cases, pop):
    alpha = 0.05

    # if cases = 0, lower bound should be 0 default
    if cases == 0:
        lower = 0
        upper = beta.ppf(1 - alpha / 2, 1, pop)
    elif cases == pop:
        lower = beta.ppf(alpha / 2, cases, pop - cases + 1)
        upper = 1
    else:
        lower = beta.ppf(alpha / 2, cases, pop - cases + 1)
        upper = beta.ppf(1 - alpha / 2, cases + 1, pop - cases)
    return lower * 100000, upper * 100000    

In [9]:
print("Lower Upper CI NaN Values Before Cleaning:", df['Lower_95__CI'].isna().sum(), df['Upper_95__CI'].isna().sum())

# calculate method only if col is NaN
for index, row in df.iterrows():
    lower, upper = clopper_pearson_exact_method(row['Cases'], row['Population'])
    if pd.isna(df.loc[index, 'Lower_95__CI']):
        df.loc[index, 'Lower_95__CI'] = lower
    if pd.isna(df.loc[index, 'Upper_95__CI']):
        df.loc[index, 'Upper_95__CI'] = upper

print("Lower Upper CI NaN Values After Cleaning:", df['Lower_95__CI'].isna().sum(), df['Upper_95__CI'].isna().sum())

Lower Upper CI NaN Values Before Cleaning: 5262 5262
Lower Upper CI NaN Values After Cleaning: 4754 4754


### If Rate = 'SC', Change Lower_95__CI, Lower_95__CI, Cases, and Rate

In [10]:
# suppressed when Rate = 'SC' or Cases is NaN
# protects confidentiality of individuals
df.loc[(df['Rate'] == 'SC') | (df['Cases'].isna()), 'Lower_95__CI'] = 'Suppressed'
df.loc[(df['Rate'] == 'SC') | (df['Cases'].isna()), 'Upper_95__CI'] = 'Suppressed'
df.loc[(df['Rate'] == 'SC'), 'Cases'] = 'Suppressed'
df.loc[(df['Rate'] == 'SC'), 'Rate'] = 'Suppressed'
df.loc[(df['Cases'].isna()), 'Rate'] = 'Suppressed'
df.loc[(df['Cases'].isna()), 'Cases'] = 'Suppressed'

/tmp/ipykernel_1873469/1000225841.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Suppressed' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[(df['Rate'] == 'SC') | (df['Cases'].isna()), 'Lower_95__CI'] = 'Suppressed'
/tmp/ipykernel_1873469/1000225841.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Suppressed' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[(df['Rate'] == 'SC') | (df['Cases'].isna()), 'Upper_95__CI'] = 'Suppressed'
/tmp/ipykernel_1873469/1000225841.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Suppressed' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[(df['Rate']

### Removing '*' From Rates

In [11]:
df.loc[(df['Rate'] != 'Suppressed'), 'Rate'] = df.loc[df['Rate'] != 'Suppressed', 'Rate'].astype(str).str.rstrip('*').astype(float)

### Add 'Other' to Sexes

In [ ]:
# add rows where sex = 'Other'
updated_data = []
for i in range(len(df)):
    updated_data.append(df.iloc[i])
    if df.loc[i, 'Sex'] == 'Male': 
        if df.loc[i, 'Cases'] != 'Suppressed' and df.loc[i, 'Rate'] != 'Suppressed' and df.loc[i, 'Lower_95__CI'] != 'Suppressed' and df.loc[i, 'Upper_95__CI'] != 'Suppressed':
            row_for_other_sex = {'Disease': df.loc[i, 'Disease'], 'County': df.loc[i, 'County'], 'Year': df.loc[i, 'Year'], 'Sex': 'Other', 'Cases': (df.loc[i + 1, 'Cases'] - df.loc[i, 'Cases'] - df.loc[i - 1, 'Cases']), 'Population': (df.loc[i + 1, 'Population'] - df.loc[i, 'Population'] - df.loc[i - 1, 'Population']), 'Rate': None, 'Lower_95__CI': None, 'Upper_95__CI': None} 
            if row_for_other_sex['Population'] > 0:
                row_for_other_sex['Rate'] = (row_for_other_sex['Cases'] * 100000)/row_for_other_sex['Population']
                lower, upper = clopper_pearson_exact_method(row_for_other_sex['Cases'], row_for_other_sex['Population'])
                row_for_other_sex['Lower_95__CI'] = lower
                row_for_other_sex['Upper_95__CI'] = upper                
        else:    
            row_for_other_sex = {'Disease': df.loc[i, 'Disease'], 'County': df.loc[i, 'County'], 'Year': df.loc[i, 'Year'], 'Sex': 'Other', 'Cases': 'Suppressed', 'Population': (df.loc[i + 1, 'Population'] - df.loc[i, 'Population'] - df.loc[i - 1, 'Population']), 'Rate': 'Suppressed', 'Lower_95__CI': 'Suppressed', 'Upper_95__CI': 'Suppressed'} 

        # mark all relevant fields as 'Cannot Exist' if the pop for Sex = Other is 0
        if row_for_other_sex['Population'] <= 0:
            row_for_other_sex['Cases'] = 'Cannot Exist'
            row_for_other_sex['Rate'] = 'Cannot Exist'
            row_for_other_sex['Lower_95__CI'] = 'Cannot Exist'
            row_for_other_sex['Upper_95__CI'] = 'Cannot Exist'
            if row_for_other_sex['Population'] < 0:
                row_for_other_sex['Population'] = 0
                df.loc[i + 1, 'Population'] = df.loc[i, 'Population'] + df.loc[i - 1, 'Population']
                df.loc[i + 1, 'Rate'] = (df.loc[i + 1, 'Cases'] * 100000)/df.loc[i + 1, 'Population']
                lower, upper = clopper_pearson_exact_method(df.loc[i + 1, 'Cases'], df.loc[i + 1, 'Population'])
                df.loc[i + 1, 'Lower_95__CI'] = lower
                df.loc[i + 1, 'Upper_95__CI'] = upper
        
        updated_data.append(pd.Series(row_for_other_sex))

final_df = pd.DataFrame(updated_data)

### Rounding

#### Rate, Lower_95__CI, and Upper_95__CI to 3 Decimals

In [ ]:
final_df.loc[(final_df['Rate'] != 'Suppressed') & (final_df['Rate'] != 'Cannot Exist'), 'Rate'] = final_df.loc[(final_df['Rate'] != 'Suppressed') & (final_df['Rate'] != 'Cannot Exist'), 'Rate'].astype(float).map(lambda x: f"{x:.3f}")
final_df.loc[(final_df['Lower_95__CI'] != 'Suppressed') & (final_df['Rate'] != 'Cannot Exist'), 'Lower_95__CI'] = final_df.loc[(final_df['Lower_95__CI'] != 'Suppressed') & (final_df['Rate'] != 'Cannot Exist'), 'Lower_95__CI'].astype(float).map(lambda x: f"{x:.3f}")
final_df.loc[(final_df['Upper_95__CI'] != 'Suppressed') & (final_df['Rate'] != 'Cannot Exist'), 'Upper_95__CI'] = final_df.loc[(final_df['Upper_95__CI'] != 'Suppressed') & (final_df['Rate'] != 'Cannot Exist'), 'Upper_95__CI'].astype(float).map(lambda x: f"{x:.3f}")
final_df.loc[(final_df['Cases'] != 'Suppressed') & (final_df['Rate'] != 'Cannot Exist'), 'Cases'] = final_df.loc[(final_df['Cases'] != 'Suppressed') & (final_df['Rate'] != 'Cannot Exist'), 'Cases'].astype(int)

### Print Final DataFrame

In [ ]:
final_df.reset_index(drop=True, inplace=True)
print("Final DataFrame")
final_df.head(20)

## Data Prediction

In [ ]:
# remove str data in numerical columns
final_df['Rate'] = pd.to_numeric(final_df['Rate'], errors='coerce')
final_df['Cases'] = pd.to_numeric(final_df['Cases'], errors='coerce')
final_df['Population'] = pd.to_numeric(final_df['Population'], errors='coerce')
final_df['Lower_95__CI'] = pd.to_numeric(final_df['Lower_95__CI'], errors='coerce')
final_df['Upper_95__CI'] = pd.to_numeric(final_df['Upper_95__CI'], errors='coerce')
final_df = final_df.dropna(subset=['Rate', 'Cases', 'Population', 'Lower_95__CI', 'Upper_95__CI'])
final_df.to_csv("cleaned.csv", index=False, header=True, sep=',')

### ARIMA Model

In [ ]:
# randomly choose disease, county, and sex
disease = np.random.choice(final_df['Disease'].unique())
county = np.random.choice(final_df['County'].unique())
sex = np.random.choice(final_df['Sex'].unique())

filtered_df = final_df[(final_df['Disease'] == disease) & (final_df['County'] == county) & (final_df['Sex'] == sex)]
filtered_df.set_index('Year', inplace=True)

# split data into train and test sets
train_data = filtered_df['Rate'][:int(0.8 * len(filtered_df))]
test_data = filtered_df['Rate'][int(0.8 * len(filtered_df)):]

# fit ARIMA model
model = auto_arima(train_data, seasonal=False, stepwise=True, suppress_warnings=True, error_action="ignore", trace=True)
print(model.summary())

# rate prediction for 2023 (the last year reported in data + 1)
n_periods = 1
prediction = model.predict(n_periods)

# plot previous rates for 2001 - 2022 and predicted rates for the following year
plt.figure(figsize=(12,6))
plt.plot(filtered_df.index, filtered_df['Rate'], label='Previous Data')
plt.plot(2023, prediction, label=f'Prediction for 2023', color='red', marker='o')
plt.title(f'Predicted Rate for {disease} in {county} for {sex} for 2023')
plt.xlabel('Year')
plt.ylabel('Rate')
plt.legend()
plt.show()

# evaluate performance w/ MAE
mae = mean_absolute_error(test_data, model.predict(n_periods=len(test_data)))
print(f"ARIMA Model Mean Absolute Error (MAE): {mae}")

# print predicted rates for following year
print(f'Predicted Rates for 2023: {prediction.tolist()}')

### Linear Regression Model

In [ ]:
# one hot encoding
encoder = OneHotEncoder()
one_hot_encoded = encoder.fit_transform(final_df[['Disease', 'County', 'Sex']]).toarray()
one_hot_encoded_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(['Disease', 'County', 'Sex']))
new_df = pd.concat([final_df.reset_index(drop=True), one_hot_encoded_df], axis=1)
print(new_df)

In [ ]:
# create/train linear regression model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
X = new_df[[col for col in new_df.columns if col.startswith("Disease_") or col.startswith("County_") or col.startswith("Sex_")]]
y = new_df['Rate']
X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.2, random_state=42)
type(X_test)
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
# create test case to predict rate given the disease, county, and sex
test_case = [
    ('Anaplasmosis', 'Alameda', 'Female'),
    ('Shigellosis', 'Mariposa', 'Male'),
    ('Cysticercosis or Taeniasis', 'Inyo', 'Other'),
    ('Yersiniosis', 'San Mateo', 'Total'),
    ('Zika Virus Infection', 'San Diego', 'Female'),
    ('Salmonellosis', 'Riverside', 'Total')
]
test_case_df = pd.DataFrame(test_case, columns=['Disease', 'County', 'Sex'])
test_case_encoded = pd.get_dummies(test_case_df, columns=['Disease', 'County', 'Sex'])
test_case_df_final = test_case_encoded.reindex(columns=X_test.columns, fill_value=0)
rate_prediction = model.predict(test_case_df_final)
print(rate_prediction)

In [ ]:
# evaluate linear regression model w/ performance metrics 
y_pred = model.predict(X_test)
print(f"Mean Squared Error: {mean_squared_error(y_test, y_pred):.2f}")
print(f"R2 Score: {r2_score(y_test, y_pred):.2f}")
print("Model Coefficients:", model.coef_)
print("Model Intercept:", model.intercept_)

In [ ]:
# calculate mean rates for every unique disease, county, and sex combination
mean_rates = final_df.groupby(['Disease', 'County', 'Sex'])['Rate'].mean().reset_index()
test_case_df = pd.DataFrame(test_case, columns=['Disease', 'County', 'Sex'])
df_merged = pd.merge(test_case_df, mean_rates, on=['Disease', 'County', 'Sex'], how='left')
arr = df_merged['Rate'].to_numpy()
print(arr)

In [ ]:
# evaluate baseline model and compare against linear regression model
mean_rate = y_train.mean()
baseline_y_pred = [mean_rate] * len(y_test)
print(f"Baseline Mean Squared Error: {mean_squared_error(y_test, baseline_y_pred):.2f}")
print(f"Baseline R2 Score: {r2_score(y_test, baseline_y_pred):.2f}")

## Data Visualization

In [ ]:
# plot rate change for a particular disease from 2001 - 2022
random_disease = np.random.choice(final_df['Disease'].unique())
df2 = final_df[(final_df['Sex'] == 'Total') & (final_df['Disease'] == random_disease)]
plt.scatter(df2['Year'], df2['Rate'], label='Rate Change')
plt.xlabel('Year')
plt.ylabel('Rate (cases per 100,000)')
plt.title(f'Rate Change From 2001 to 2022 for {random_disease.capitalize()}')
year = df2['Year']
rate = df2['Rate']
linear_fit = np.polyfit(year, rate, 1)
trendline = np.poly1d(linear_fit)
plt.plot(df2['Year'], trendline(df2['Year']), label='Trendline', color='red')
plt.show()

In [ ]:
# plot rate change for a particular disease for a specific county from 2001 - 2022
random_disease = np.random.choice(final_df['Disease'].unique())
random_county = np.random.choice(final_df['County'].unique())
df2 = final_df[(final_df['Sex'] == 'Total') & (final_df['Disease'] == random_disease) & (final_df['County'] == random_county)]
plt.scatter(df2['Year'], df2['Rate'], label='Rate Change')
plt.xlabel('Year')
plt.ylabel('Rate (cases per 100,000)')
plt.title(f'Rate Change From 2001 to 2022 for {random_disease.capitalize()} within {random_county.capitalize()} County')
year = df2['Year']
rate = df2['Rate']
linear_fit = np.polyfit(year, rate, 1)
trendline = np.poly1d(linear_fit)
plt.plot(df2['Year'], trendline(df2['Year']), label='Trendline', color='red')
plt.show()

## Database

In [ ]:
conn = sqlite3.connect('california_diseases')
cursor = conn.cursor()

cursor.execute('DROP TABLE IF EXISTS Disease')
cursor.execute('DROP TABLE IF EXISTS CountyCases')

# Disease table
cursor.execute('''
    CREATE TABLE Disease (
        name VARCHAR(25) PRIMARY KEY,
        link MEDIUMTEXT
    )
''')

# CountyCases table
cursor.execute('''
    CREATE TABLE CountyCases (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        disease VARCHAR(25),
        county VARCHAR(25) NOT NULL,
        year INTEGER NOT NULL,
        sex VARCHAR(6) NOT NULL,
        population INTEGER NOT NULL,
        cases TEXT,
        rate TEXT,
        lower_95_ci TEXT,
        upper_95_ci TEXT,
        FOREIGN KEY (disease) REFERENCES Disease (name)
    )
''')

### Describe Tables

In [ ]:
# describe Disease table
cursor.execute("PRAGMA table_info(Disease);")
cursor.fetchall()

In [ ]:
# describe CountyCases table
cursor.execute("PRAGMA table_info(CountyCases);")
cursor.fetchall()

### Insert Into Disease and County Tables

In [ ]:
unique_diseases = final_df['Disease'].unique()

# insert disease into Disease table
cursor.executemany('INSERT OR IGNORE INTO Disease (name) VALUES (?)', [(disease,) for disease in unique_diseases])
conn.commit()

county_cases = []
# append data from each row and col into county_cases
for _, row in final_df.iterrows():
    county_cases.append((
        row['Disease'], row['County'], row['Year'], row['Sex'], row['Population'],
        row['Cases'], row['Rate'], row['Lower_95__CI'], row['Upper_95__CI']
    ))

# insert multiple records into CountyCases table
cursor.executemany('''
    INSERT INTO CountyCases (disease, county, year, sex, population, cases, rate, lower_95_ci, upper_95_ci)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
''', county_cases)
conn.commit()

# show first 10 records of Disease table
cursor.execute("SELECT * FROM Disease LIMIT 10")
rows = cursor.fetchall()
for row in rows:
    print(row)

### Ensure CountCases Table Insertions Are Correct

In [ ]:
# print 10 random records of CountyCases table
cursor.execute('''SELECT * FROM CountyCases ORDER BY RANDOM() LIMIT 10''')
rows = cursor.fetchall()
for row in rows:
    print(row)

In [ ]:
# print 10 random cases where cases > 0
cursor.execute('''SELECT * FROM CountyCases WHERE cases > 0 ORDER BY RANDOM() LIMIT 10''')
rows = cursor.fetchall()
for row in rows:
    print(row)

### Questions

#### Which 10 diseases have the highest total case count? 

In [ ]:
# gets the cumulative case count for each disease, sorting by descending case count
cursor.execute('''
    SELECT disease, SUM(CAST(cases as INT)) AS sum_case 
    FROM CountyCases
    WHERE year BETWEEN 2001 AND 2022
    GROUP BY disease
    ORDER BY sum_case DESC
    LIMIT 10
''')

results = cursor.fetchall()
for row in results:
    disease, sum_case = row
    print(f"{disease}: {sum_case} cases")

#### What are the 10 most common diseases for females? Males? Others?

In [ ]:
# gets the cumulative case count for each sex, sorting by descending case count
queries = [
    'SELECT sex, disease, SUM(CAST(cases AS INT)) AS max_cases FROM CountyCases WHERE year BETWEEN 2001 AND 2022 AND sex = "Female" GROUP BY disease ORDER BY max_cases DESC LIMIT 10',
    'SELECT sex, disease, SUM(CAST(cases AS INT)) AS max_cases FROM CountyCases WHERE year BETWEEN 2001 AND 2022 AND sex = "Male" GROUP BY disease ORDER BY max_cases DESC LIMIT 10',
    'SELECT sex, disease, SUM(CAST(cases AS INT)) AS max_cases FROM CountyCases WHERE year BETWEEN 2001 AND 2022 AND sex = "Other" GROUP BY disease ORDER BY max_cases DESC LIMIT 10',
]

for query in queries:
    cursor.execute(query)
    results = cursor.fetchall()
    for row in results:
        sex, disease, max_cases = row
        print(f"{sex}: {disease}, {max_cases} cases")
    print()

#### Which 5 counties have the highest disease rate for the top 5 common diseases in 2022?

In [ ]:
# CommonDisease: gets 5 diseases with the highest total cases and orders by descending
# TopCounties: ranks counties by descending rate for each disease
    # PARTITION BY: divides the rows into partitions by disease name, orders by descending rate
    # ROW_NUMBER(): assigns each row in each partition a sequential int num
# gets the top 5 counties for each disease that have the highest rates
cursor.execute('''
WITH CommonDiseases AS (
    SELECT disease, SUM(CAST(cases AS INT)) AS sum_case
    FROM CountyCases
    WHERE year BETWEEN 2001 AND 2022
    GROUP BY disease
    ORDER BY sum_case DESC
    LIMIT 5
),
TopCounties AS (
    SELECT cd.disease, cc.county, 
        MAX(CAST(cc.rate AS REAL)) AS max_rate,
        ROW_NUMBER() OVER (PARTITION BY cd.disease ORDER BY MAX(CAST(cc.rate AS REAL)) DESC) AS rank,
        cd.sum_case
    FROM (
        SELECT DISTINCT disease, county, rate
        FROM CountyCases
        WHERE year = 2022
    ) AS cc
    JOIN CommonDiseases cd ON cc.disease = cd.disease
    GROUP BY cd.disease, cc.county, cd.sum_case
)
SELECT disease, county, ROUND(max_rate, 3) AS max_rate
FROM TopCounties
WHERE rank <= 5
ORDER BY sum_case DESC, disease, rank
''')

results = cursor.fetchall()
for i, row in enumerate(results):
    disease, county, max_rate = row
    print(f"{disease}: {county} County, {max_rate:.3f}")  
    if (i + 1) % 5 == 0:
        print()

## Web Scraping

In [ ]:
# search wikipedia using search results or direct page match
def search_wiki(disease):
    search_url = f"https://en.wikipedia.org/w/index.php?search={disease.replace(' ', '+')}"
    response = requests.get(search_url)
    soup = BeautifulSoup(response.text, "html.parser")

    # get first search result
    first = soup.find("li", class_="mw-search-result")
    if first:
        link = "https://en.wikipedia.org" + first.find("a")["href"]
        return link
    else:
        # see if there's a direct page match
        direct_match = soup.find("h1", class_="firstHeading")
        if direct_match and "Search results" not in direct_match.text:
            return search_url
        return None

In [ ]:
# get wiki link for every disease
unique_diseases = final_df['Disease'].unique()
for disease in unique_diseases:
    link = search_wiki(disease)
    if link:
        cursor.execute('UPDATE Disease SET link = ? WHERE name = ?', (link, disease))
conn.commit()

cursor.execute("SELECT * FROM Disease")
rows = cursor.fetchall()
for row in rows:
    print(row)

In [ ]:
# close connection
conn.close()